In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [125]:
compras = (pd.concat
                ([
                pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'),
                productos_sin_compra
               ])
        )


cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'].dt.normalize() - pd.Timedelta(days=5)

In [127]:
costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

# Checks

In [128]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Todo correcto con check1


In [129]:
check2 = costo_venta[costo_venta['product_price'].isna()][['product_id', 'product_name_x']]
print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 21 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10272,28586,Anticipo
13415,28638,Servicios de Facturación
18005,4,Anticipo (PdV)


# Pruebas

In [133]:
product_id_to_check = 13811

wep_costo_venta = (costo_venta
    .loc[costo_venta['product_id'] == product_id_to_check]
        [['name', 'invoice_date', 'salesperson_name', 'quantity',
        'price_subtotal', 'product_price', 'order_id', 'order_name',]]
)

wep_compras = compras.loc[(compras['product_id_pp'] == product_id_to_check)]


writer = pd.ExcelWriter('comparativa_costo_venta.xlsx', engine="openpyxl")

wep_costo_venta.to_excel(writer, sheet_name='costo_venta')
wep_compras.to_excel(writer, sheet_name='compras')

writer.close()
writer.handles = None

In [134]:
wep_costo_venta

,name,invoice_date,salesperson_name,quantity,price_subtotal,product_price,order_id,order_name
402,F1-VS/2024/00015,2024-01-04,Mayra Angelica Parada Manjarrez,6.00,"4,505.40",443.34,107,P00096 (B 83302)
1562,F1-VS/2024/00038,2024-01-06,Mayra Angelica Parada Manjarrez,3.00,"2,163.00",474.14,164,P00153 (MPIFCO 622)
1709,F1-VS/2024/00080,2024-01-08,Irma Carvajal Flores,5.00,"3,400.00",443.34,198,P00187 (B 83392)
2179,F1-VS/2024/00097,2024-01-09,Irma Carvajal Flores,6.00,"4,038.00",443.34,198,P00187 (B 83392)
2825,F1-VS/2024/00127,2024-01-11,Mayra Angelica Parada Manjarrez,6.00,"4,269.60",443.34,198,P00187 (B 83392)
3395,F1-VS/2024/00152,2024-01-12,Irma Carvajal Flores,2.00,"1,440.00",443.34,259,P00248 (B 83459)
4883,F1-VS/2024/00203,2024-01-17,Irma Carvajal Flores,4.00,"2,560.00",443.34,309,P00298 (B 83528)
4924,F1-VS/2024/00207,2024-01-17,Irma Carvajal Flores,14.00,"10,010.00",443.34,309,P00298 (B 83528)
5233,F1-VS/2024/00191,2024-01-17,Irma Carvajal Flores,20.00,"12,800.00",443.34,309,P00298 (B 83528)
5913,F1-VS/2024/00244,2024-01-19,Irma Carvajal Flores,18.00,"11,250.00",456.90,377,P00366 (SJCFCO 2008)


In [135]:
wep_compras

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
17,19070,3117,P03103 (MPIFCO 12908),2024-06-25 22:19:17,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,5.00,469.83,False,MPIFCO 12908,2024-06-25,Alexa Yadira Mazariegos Zunun,Alexa Yadira Mazariegos Zunun,2024-06-20
941,17447,2886,P02872 (B 86851),2024-06-11 23:43:35,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,5.00,514.55,False,B 86851,2024-06-11,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-06
1407,16584,2779,P02766 (SJCFCO 14666),2024-06-05 23:56:24,5326,Madereria El Pino De Los Cabos,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,80.00,443.97,False,SJCFCO 14666,2024-06-05,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez,2024-05-31
1649,16164,2718,P02705 (B 86651),2024-06-04 00:19:47,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,100.00,443.34,False,B 86651,2024-06-03,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores,2024-05-30
1651,16160,2717,P02704 (MPIFCO 11393),2024-06-04 00:17:31,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,10.00,456.90,False,MPIFCO 11393,2024-06-03,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-05-30
2097,15405,2633,P02620 (MPIFCO 11004),2024-05-29 22:53:25,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,6.00,456.90,False,MPIFCO 11004,2024-05-29,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores,2024-05-24
2131,15339,2628,P02615 (B 86475),2024-05-28 23:56:28,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,1.00,443.34,False,B 86475,2024-05-28,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-05-23
2192,15244,2598,P02585 (SJCFCO 13661),2024-05-25 15:41:29,5326,Madereria El Pino De Los Cabos,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,3.00,456.90,False,SJCFCO 13661,2024-05-25,Gretel Gómez Alcázar,Yamilet Blanco Salas,2024-05-20
2368,14910,2583,P02570 (A 67401),2024-05-24 20:59:16,5343,Irma Rodriguez Montoya,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,3.00,469.83,False,A 67401,2024-05-24,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez,2024-05-19
2517,14642,2545,P02532 (SJCFCO 13438),2024-05-24 16:04:05,5326,Madereria El Pino De Los Cabos,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,1.00,456.90,False,SJCFCO 13438,2024-05-22,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-05-19


In [15]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [16]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]

In [108]:
prod_varios_prov = compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']].sort_values('partner_id').groupby('product_id_pp').agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
prod_varios_prov.columns = ['product_name', 'mean', 'first']
prod_varios_prov['diff'] = prod_varios_prov['mean']  == prod_varios_prov['first']
prod_varios_prov.loc[prod_varios_prov['diff'] == False]

,product_name,mean,first,diff
product_id_pp,,,,
7480,"[2538] Rondana Plana 1/4"" C#181 *2538*","5,337.00",5221,False
7481,[2540] Rondana Plana 3/16 C#184 *2540*,"5,293.50",5221,False
7483,"[6528] Pija Autorr. C/Rondana 8""*1/2"" Galv. *6...","5,345.29",5221,False
7510,"[5388] Tuerca Hexagonal 1/4"" Hncz (5600/Ct) C...","5,329.75",5221,False
7511,"[10179] Taq. Plastico El-Pro 1/4"" Blanco TQ-01...","5,328.20",5313,False
...,...,...,...,...
29236,[4413] Brida Flexible Coflex PB-300 Larga *4413*,"5,326.33",5309,False
29241,[4426] Color Cemento Negro *4426*,"5,236.50",5143,False
29255,[4497] Guantes De Corte De Nitrilo 48-22-8902 ...,"5,179.80",5128,False


In [109]:
prod_varios_prov_ids = prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp']
prod_varios_prov_ids

0       7480
1       7481
2       7483
3       7510
4       7511
       ...  
206    29236
207    29241
208    29255
209    29436
210    29452
Name: product_id_pp, Length: 211, dtype: Int64

In [110]:
compras[compras['product_id_pp'].isin(prod_varios_prov_ids)]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora
2,19086,3127,P03113 (SJCFCO 16603),2024-06-21,5326,Madereria El Pino De Los Cabos,13697,[13748] Triplay Pino 1/2 4'*8' 0/Caras *13748*,80.00,383.62,False,SJCFCO 16603,2024-06-26,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas
3,19087,3127,P03113 (SJCFCO 16603),2024-06-21,5326,Madereria El Pino De Los Cabos,13671,"[13902] Polin 4"" X 4"" X 12' *13902*",40.00,387.93,False,SJCFCO 16603,2024-06-26,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas
4,19083,3126,P03112 (FC-LPZ1-83952),2024-06-21,5285,Surtidora Mexicana Frontera,10826,[3687] Pasador Ph #36-C #133 *3687*,12.00,54.39,False,FC-LPZ1-83952,2024-06-25,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez
7,19081,3124,P03110 (SJCFCO 16545),2024-06-21,5326,Madereria El Pino De Los Cabos,14184,[90057] Tablon americano 2*12*10 *90057*,1.00,758.62,False,SJCFCO 16545,2024-06-26,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez
8,19080,3123,P03109 (LC 9742),2024-06-21,5181,Vector De Baja California,28214,[12671] Tablaroca Verde anti-moho USG *12671*,10.00,423.87,False,LC 9742,2024-06-26,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,<NA>,<NA>,SAE,2023-10-11,<NA>,NaN,14016,Maletin D/Broca SDS-Plus MK D-31734 13pz *8635*,NaN,599.00,NaN,NaN,NaT,NaN,NaN
6085,<NA>,<NA>,SAE,2023-11-18,<NA>,NaN,14671,Malla Sombra 2x100m 90% Beige Raschel *96679*,NaN,"3,620.70",NaN,NaN,NaT,NaN,NaN
6768,<NA>,<NA>,SAE,2023-01-14,<NA>,NaN,27422,Silicon Cafe DAP Alex Plus 300 ml *2468*,NaN,82.76,NaN,NaN,NaT,NaN,NaN
7262,<NA>,<NA>,SAE,2023-11-03,<NA>,NaN,28253,Detector Tipo Lapiz 1AC-A1-II Mca Fluke *11508*,NaN,828.25,NaN,NaN,NaT,NaN,NaN
